In [1]:
import pandas as pd
import numpy as np
import pickle
from file_operations import file_methods
from application_logging import logger
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report,accuracy_score,roc_curve,roc_auc_score,confusion_matrix
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
%matplotlib inline

In [2]:
data = pd.read_csv("Training_Batch_Files/creditCardFraud_28011961_120210.csv")
data.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,20000.0,2,2,1,24,2,2,-1,-1,-2,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,120000.0,2,2,2,26,-1,2,0,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,90000.0,2,2,2,34,0,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,50000.0,2,2,1,37,0,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,50000.0,1,2,1,57,-1,0,-1,0,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [3]:
x = data.drop(columns="default.payment.next.month")
x.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,20000.0,2,2,1,24,2,2,-1,-1,-2,...,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0
1,120000.0,2,2,2,26,-1,2,0,0,0,...,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0
2,90000.0,2,2,2,34,0,0,0,0,0,...,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0
3,50000.0,2,2,1,37,0,0,0,0,0,...,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0
4,50000.0,1,2,1,57,-1,0,-1,0,0,...,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0


In [4]:
x.iloc[790:800]

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
790,30000.0,2,2,2,51,1,2,2,0,0,...,25761.0,25964.0,26507.0,27249.0,1800.0,0.0,928.0,960.0,1181.0,967.0
791,70000.0,1,1,2,27,1,-1,-1,-1,-2,...,19.0,-254.0,-527.0,1096.0,600.0,19.0,0.0,0.0,1896.0,0.0
792,410000.0,1,1,2,32,-1,-1,-1,-2,-2,...,-281.0,-281.0,-281.0,-281.0,5400.0,0.0,0.0,0.0,0.0,0.0
793,280000.0,2,3,1,40,1,-2,-2,-2,-2,...,-288.0,-288.0,-288.0,2124.0,0.0,0.0,0.0,0.0,2412.0,0.0
794,50000.0,2,1,1,34,0,0,0,0,0,...,48282.0,18299.0,19323.0,20329.0,2320.0,1282.0,3299.0,1323.0,1329.0,1114.0
795,100000.0,2,2,1,30,0,0,0,0,0,...,75004.0,76705.0,78168.0,80347.0,2800.0,2700.0,5400.0,3500.0,3500.0,3200.0
796,230000.0,2,1,2,29,-2,-2,-2,-2,-2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
797,50000.0,2,1,2,22,0,0,0,-1,-1,...,5385.0,7781.0,0.0,0.0,1372.0,1275.0,7900.0,0.0,0.0,0.0
798,100000.0,2,1,2,48,2,2,2,2,2,...,18158.0,18365.0,17932.0,18147.0,3164.0,800.0,800.0,0.0,662.0,3000.0
799,210000.0,2,2,1,35,0,0,0,0,0,...,34232.0,32331.0,30337.0,28012.0,2000.0,2000.0,2444.0,1342.0,1038.0,2000.0


In [5]:
x.columns

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'],
      dtype='object')

In [6]:
y = data['default.payment.next.month']
y.head()

0    1
1    1
2    0
3    0
4    0
Name: default.payment.next.month, dtype: int64

In [7]:
# spliting data into train and test
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state=100)

In [8]:
x_train

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
795,100000.0,2,2,1,30,0,0,0,0,0,...,75004.0,76705.0,78168.0,80347.0,2800.0,2700.0,5400.0,3500.0,3500.0,3200.0
23652,20000.0,2,1,1,37,1,-2,-1,2,2,...,10887.0,10413.0,-245.0,-245.0,1575.0,11000.0,0.0,0.0,0.0,5100.0
9038,160000.0,2,2,1,31,0,0,0,0,0,...,76660.0,79825.0,82951.0,85168.0,5000.0,10000.0,5000.0,5000.0,4000.0,4000.0
11588,280000.0,1,2,2,30,-1,-1,-1,-1,-1,...,651.0,651.0,651.0,651.0,1650.0,651.0,651.0,651.0,651.0,0.0
7422,60000.0,2,1,2,25,0,0,0,2,2,...,27050.0,26317.0,28200.0,28762.0,1414.0,2678.0,0.0,2310.0,1173.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12119,20000.0,2,1,2,22,1,2,2,0,0,...,11164.0,11395.0,11927.0,9186.0,1100.0,0.0,289.0,594.0,0.0,50.0
24423,10000.0,1,2,1,36,2,0,0,0,0,...,9582.0,8986.0,9691.0,9889.0,2000.0,1300.0,1200.0,1000.0,1000.0,569.0
14147,100000.0,2,2,2,27,0,0,0,0,0,...,90357.0,89209.0,84983.0,81484.0,4171.0,4044.0,3420.0,3300.0,3886.0,2730.0
23320,80000.0,2,1,1,29,-1,-1,-1,-1,0,...,1872.0,746.0,746.0,4424.0,358.0,1872.0,746.0,0.0,4424.0,0.0


In [9]:
x_train.shape

(24000, 23)

In [10]:
y_train.shape

(24000,)

In [11]:
x_test.shape

(6000, 23)

In [12]:
y_test.shape

(6000,)

In [13]:
# Initializing standard scaler object
train_scaler = StandardScaler()
test_scaler = StandardScaler()

In [14]:
scaler=train_scaler.fit(x_train)

In [15]:
pickle.dump(train_scaler, open('models/scaler/scaler.sav', 'wb'))

In [16]:
scaled_train_data=scaler.transform(x_train)
scaled_test_data=test_scaler.fit_transform(x_test)

In [17]:
scaler.transform([[100000,2,2,1,30,0,0,0,0,0,0,97062,74352,75004,76705,78168,80347,2800,2700,5400,3500,3500,3200]])

/home/swapnil/anaconda3/envs/project/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[-0.52121838,  0.80949773,  0.18492014, -1.05575343, -0.5959469 ,
         0.01872585,  0.11662224,  0.14377102,  0.19721976,  0.24377449,
         0.26145029,  0.61818495,  0.34982064,  0.40094891,  0.51503605,
         0.6207635 ,  0.6904972 , -0.16840884, -0.13341025,  0.00879938,
        -0.08241696, -0.08868622, -0.11518792]])

In [18]:
scaled_train_df=pd.DataFrame(data=scaled_train_data, columns=x_train.columns)

In [19]:
scaled_test_df=pd.DataFrame(data=scaled_test_data, columns=x_test.columns, index=x_test.index)

In [20]:
scaled_train_df.iloc[0]

LIMIT_BAL   -0.521218
SEX          0.809498
EDUCATION    0.184920
MARRIAGE    -1.055753
AGE         -0.595947
PAY_0        0.018726
PAY_2        0.116622
PAY_3        0.143771
PAY_4        0.197220
PAY_5        0.243774
PAY_6        0.261450
BILL_AMT1    0.618185
BILL_AMT2    0.349821
BILL_AMT3    0.400949
BILL_AMT4    0.515036
BILL_AMT5    0.620763
BILL_AMT6    0.690497
PAY_AMT1    -0.168409
PAY_AMT2    -0.133410
PAY_AMT3     0.008799
PAY_AMT4    -0.082417
PAY_AMT5    -0.088686
PAY_AMT6    -0.115188
Name: 0, dtype: float64

In [21]:
scaled_train_df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,-0.521218,0.809498,0.184920,-1.055753,-0.595947,0.018726,0.116622,0.143771,0.197220,0.243774,...,0.400949,0.515036,0.620763,0.690497,-0.168409,-0.133410,0.008799,-0.082417,-0.088686,-0.115188
1,-1.136449,0.809498,-1.081295,-1.055753,0.160668,0.906908,-1.553386,-0.692918,1.917785,2.019101,...,-0.519449,-0.509328,-0.665178,-0.654181,-0.239433,0.211102,-0.291994,-0.305623,-0.316181,-0.011962
2,-0.059796,0.809498,0.184920,-1.055753,-0.487859,0.018726,0.116622,0.143771,0.197220,0.243774,...,0.424721,0.563247,0.699203,0.770936,-0.040856,0.169594,-0.013482,0.013243,-0.056187,-0.071724
3,0.863049,-1.235334,0.184920,0.856555,-0.595947,-0.869457,-0.718382,-0.692918,-0.663063,-0.643889,...,-0.666387,-0.660174,-0.650484,-0.639231,-0.235084,-0.218459,-0.255732,-0.264107,-0.273867,-0.289042
4,-0.828833,0.809498,-1.081295,0.856555,-1.136386,0.018726,0.116622,0.143771,1.917785,2.019101,...,-0.287430,-0.263575,-0.198691,-0.170199,-0.248767,-0.134323,-0.291994,-0.158307,-0.239937,-0.289042


## Gaussian Naive-Bayes

In [22]:
gnb=GaussianNB()

In [23]:
gnb.fit(scaled_train_df,y_train)

GaussianNB()

In [24]:
gnbPred_y = gnb.predict(scaled_test_df)

In [25]:
acGNB = accuracy_score(y_test,gnbPred_y)
acGNB

0.752

In [26]:
param_grid = {"var_smoothing": [1e-9,0.1, 0.001, 0.5,0.05,0.01,1e-8,1e-7,1e-6,1e-10,1e-11]}
#Creating an object of the Grid Search class
grid = GridSearchCV(estimator=gnb, param_grid=param_grid, cv=5,  verbose=3)

In [27]:
grid.fit(scaled_train_data,y_train)

Fitting 5 folds for each of 11 candidates, totalling 55 fits
[CV 1/5] END ...............var_smoothing=1e-09;, score=0.733 total time=   0.0s
[CV 2/5] END ...............var_smoothing=1e-09;, score=0.725 total time=   0.0s
[CV 3/5] END ...............var_smoothing=1e-09;, score=0.750 total time=   0.0s
[CV 4/5] END ...............var_smoothing=1e-09;, score=0.709 total time=   0.0s
[CV 5/5] END ...............var_smoothing=1e-09;, score=0.743 total time=   0.0s
[CV 1/5] END .................var_smoothing=0.1;, score=0.765 total time=   0.0s
[CV 2/5] END .................var_smoothing=0.1;, score=0.766 total time=   0.0s
[CV 3/5] END .................var_smoothing=0.1;, score=0.770 total time=   0.0s
[CV 4/5] END .................var_smoothing=0.1;, score=0.759 total time=   0.0s
[CV 5/5] END .................var_smoothing=0.1;, score=0.774 total time=   0.0s
[CV 1/5] END ...............var_smoothing=0.001;, score=0.734 total time=   0.0s
[CV 2/5] END ...............var_smoothing=0.001;

GridSearchCV(cv=5, estimator=GaussianNB(),
             param_grid={'var_smoothing': [1e-09, 0.1, 0.001, 0.5, 0.05, 0.01,
                                           1e-08, 1e-07, 1e-06, 1e-10, 1e-11]},
             verbose=3)

In [28]:
grid.best_estimator_

GaussianNB(var_smoothing=0.5)

In [29]:
# retrianing after hyperparamter tuning
gnb_new=GaussianNB(var_smoothing=0.5)

In [30]:
gnb_new.fit(scaled_train_df,y_train)

GaussianNB(var_smoothing=0.5)

In [31]:
gnbPred_yNew = gnb_new.predict(scaled_test_df)

In [32]:
acGNB_New = accuracy_score(y_test,gnbPred_yNew)

In [33]:
acGNB_New

0.7996666666666666

## XGBoost

In [34]:
xb = xgb.XGBClassifier(objective='binary:logistic')

In [35]:
xb.fit(scaled_train_df,y_train)

/home/swapnil/anaconda3/envs/project/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:19:07] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [36]:
xgb_pred = xb.predict(scaled_test_df)
xgb_pred

array([0, 1, 0, ..., 0, 0, 0])

In [37]:
ac_xgb = accuracy_score(y_test,xgb_pred)
ac_xgb

0.8091666666666667

In [38]:
# param_grid_xgboost = {
#     "n_estimators": [50,100, 130],
#     "max_depth": range(3, 11, 1),
#     "random_state":[0,50,100]
# }

In [39]:
# grid = GridSearchCV(xb,param_grid_xgboost,verbose=3,n_jobs=-1)

In [40]:
# grid.fit(scaled_train_data,y_train)

In [41]:
# grid.best_estimator_

In [42]:
xgb_new = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1,
              gamma=0, gpu_id=-1,learning_rate=0.300000012,
              max_delta_step=0, max_depth=3, min_child_weight=1, missing=1,
              n_estimators=50, n_jobs=8,
              num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', verbosity=1)

In [43]:
xgb_new.fit(scaled_train_df,y_train)

/home/swapnil/anaconda3/envs/project/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:19:10] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=3, min_child_weight=1, missing=1,
              monotone_constraints='()', n_estimators=50, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=1)

In [44]:
xgb_new_pred = xgb_new.predict(scaled_test_df)
xgb_new_pred

array([0, 0, 0, ..., 0, 0, 0])

In [45]:
ac_xgb_new = accuracy_score(y_test,xgb_new_pred)
ac_xgb_new

0.823

In [46]:
dt_model = DecisionTreeClassifier()
dt_model.fit(scaled_train_df,y_train)

DecisionTreeClassifier()

In [47]:
dt_pred = dt_model.predict(scaled_test_df)
dt_pred

array([0, 1, 0, ..., 0, 1, 0])

In [48]:
ac_dt = accuracy_score(y_test,dt_pred)
ac_dt

0.686

In [49]:
#for DecisionTreeClassifier
grid_pram = {"criterion":['gini','entropy'],
             "splitter":['best','random'],
             "max_depth" : range(2,40,1),
             "min_samples_split":range(2,10 ,1),
             "min_samples_leaf":range(1,10,1),
             'ccp_alpha':np.random.rand(20)
             }

In [50]:
grid_ccp = GridSearchCV(estimator=dt_model, param_grid=grid_pram , cv = 5 ,scoring='roc_auc', n_jobs=-1,verbose=3)

In [51]:
# grid_ccp.fit(scaled_train_df,y_train)

In [52]:
# grid_ccp.best_params_

In [53]:
# dt_model_new = DecisionTreeClassifier(criterion="entropy",
                                   max_depth=2,
                                   min_samples_leaf=1,
                                   min_samples_split=2,
                                   splitter='best',
                                   ccp_alpha=0.06691925422557021)

IndentationError: unexpected indent (1155032572.py, line 2)

In [ ]:
# dt_model_new.fit(scaled_train_df,y_train)

In [ ]:
# dt_pred_new = dt_mode_new.predict(scaled_test_df)
# dt_pred_new

In [ ]:
# ac_dt_new = accuracy_score(y_test,dt_pred)
# ac_dt_new